In [ ]:
import openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] =""

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.9/411.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
import jieba
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from typing import List

In [ ]:
loader = CSVLoader('law_data_3k.csv')
documents = loader.load()
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
docs = text_splitter.split_documents(documents)

In [ ]:
docs[0]

Document(metadata={'source': 'law_data_3k.csv', 'row': 0}, page_content='data: 民法商法-农民专业合作社法2017-12-27：第十五条 农民专业合作社章程应当载明下列事项：,（一）名称和住所；,（二）业务范围；,（三）成员资格及入社、退社和除名；,（四）成员的权利和义务；,（五）组织机构及其产生办法、职权、任期、议事规则；,（六）成员的出资方式、出资额，成员出资的转让、继承、担保；,（七）财务管理和盈余分配、亏损处理；,（八）章程修改程序；,（九）解散事由和清算办法；,（十）公告事项及发布方式；,（十一）附加表决权的设立、行使方式和行使范围；,（十二）需要载明的其他事项。')

In [ ]:
OpenAIEmbeddings()

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7e7194e9eaa0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e7194056ec0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00


In [ ]:
vectorstore = FAISS.from_documents(docs, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="mmr")

In [ ]:
retriever.invoke("借款人去世， 继承人是否应履行偿还义务？")

[Document(id='2d4551a0-6e8c-4dfd-8da9-72923872e4b7', metadata={'source': 'law_data_3k.csv', 'row': 1234}, page_content='data: 他的家人是否有义务替他还高利贷\n当然没有义务了，除非家人继承了她的遗产，那么这部分遗产首先要用来还欠款。'),
 Document(id='90c3bf2f-5787-467a-8a01-0aa3ac766f28', metadata={'source': 'law_data_3k.csv', 'row': 1148}, page_content='data: 公证处公证，老人去世后这份遗嘱是否有效\n老人生前留下遗嘱没有公证老人去世后遗嘱能在法律上生效。《中华人民共和国继承法》第十六条公民可以依照本法规定立遗嘱处分个人财产，并可以指定遗嘱执行人。公民可以立遗嘱将个人财产指定由法定继承人的一人或者数人继承。公民可以立遗嘱将个人财产赠给国家、集体或者法定继承人以外的人。'),
 Document(id='3614d4b0-06f6-4c8b-aaf7-511a1811469d', metadata={'source': 'law_data_3k.csv', 'row': 674}, page_content='data: 借款人到期一直不还钱，可否另行要求精神赔偿\n一般不能请求精神赔偿，除非你能证明不还钱给你带来了极大的精神痛苦当遇到欠钱不还的情况，作为债权人首先不要着急，而是应该马上找到律师询问，然后搜集证据，最后上诉法*。如果他们还敢不执行法*判决，即使是被判坐牢也要还清，而不是一厢情愿的认为坐牢抵债。拒不执行判决会有什么后果？根据《刑法》第三百一十三条规定：对人*法*的判决、裁定有能力执行而拒不执行，情节严重的，处三年以下有期徒刑、拘役或者罚金；情节特别严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯前款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照前款的规定处罚。下列情形属于“有能力执行而拒不执行，情节严重”的情形：被执行人隐藏、转移、故意毁损财产或者无偿转让财产、以明显不合理的低价转让财产，致使判决、裁定无法执行的；担保人或者被执行人隐藏、转移、故意毁损或者转让已向人*法*提供担

In [ ]:
retriever.invoke("如何通过法律手段应对民间借贷纠纷？")

[Document(id='d403f898-3bae-48ad-8969-56882e5a644d', metadata={'source': 'law_data_3k.csv', 'row': 931}, page_content='data: 公职人员放高利贷和从事营利性活动应受到法律怎样处理\n民间借贷属于民事行为，受到民法和合同法的约束和保护。但根据《合同法》第二百一十一条规定：“自然人之间的借款合同约定支付利息的，借款的利率不得违反国家有关限制借款利率的规定”。依据《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》，借贷双方约定的利率未超过年利率24%，出借人有权请求借款人按照约定的利率支付利息；但如果借贷双方约定的利率超过年利率36%，则超过年利率36%部分的利息应当被认定无效。《最高人民法院关于审理民间借贷案件适用法律若干问题的规定》第二十六条借贷双方约定的利率未超过年利率24%，出借人请求借款人按照约定的利率支付利息的，人民法院应予支持。借贷双方约定的利率超过年利率36%，超过部分的利息约定无效。借款人请求出借人返还已支付的超过年利率36%部分的利息的，人民法院应予支持。'),
 Document(id='3e3160f0-92fc-4046-8aca-d64e4aad309c', metadata={'source': 'law_data_3k.csv', 'row': 659}, page_content='data: 高级人民法院民间借贷纠纷应该怎么处理\n高院一般可能不会处理这种案件。这种民事案件顶多到中院就可以解决'),
 Document(id='f5af9431-0ab7-4eb6-b891-7a23ce9c18fe', metadata={'source': 'law_data_3k.csv', 'row': 609}, page_content='data: 小额借款合同纠纷怎么解决，我不还钱要坐牢吗\n基本是属于民事纠纷的，不会涉及刑事责任，但是建议积极想办法偿还的，如果拒不偿还，对方是可以起诉解决的，在法院判决后，仍不执行的话，是可以申请强制执行的，同时还会影响个人征信等'),
 Document(id='d9adaa91-1546-4b2b-9997-6cf136b6b572', metadata={'source'

In [ ]:
retriever.invoke("没有赡养老人就无法继承财产吗？")

[Document(id='f84c8e1b-5f65-40cc-8c63-a2680cb0e603', metadata={'source': 'law_data_3k.csv', 'row': 637}, page_content='data: 可不可以放弃母亲的遗产也不赡养她,从小没有养过我，连看都没有看过我一次\n可不可以放弃母亲的遗产也不赡养她,从小没有养过我，连看都没有看过我一次\n继承开始后，继承人放弃继承的，应当在遗产处理前，作出放弃继承的表示。没有表示的，视为接受继承。子女有赡养父母的义务，不能通过协议方式排除赡养父母，以协议方式排除赡养义务，协议没有法律效力。另外，子女不履行赡养义务时，无劳动能力的或生活困难的父母，有要求子女付给赡养费的权利。相关法律规定：《中华人民共和国婚姻法》第二十一条父母对子女有抚养教育的义务；子女对父母有赡养扶助的义务。父母不履行抚养义务时，未成年的或不能独立生活的子女，有要求父母付给抚养费的权利。子女不履行赡养义务时，无劳动能力的或生活困难的父母，有要求子女付给赡养费的权利'),
 Document(id='9d24578f-1dc8-49d5-8ded-5013341c9e55', metadata={'source': 'law_data_3k.csv', 'row': 596}, page_content='data: 民法典-继承编：第一千一百六十条 无人继承又无人受遗赠的遗产，归国家所有，用于公益事业；死者生前是集体所有制组织成员的，归所在集体所有制组织所有。'),
 Document(id='e8a2cf40-89ab-451b-8bd1-a187a5460568', metadata={'source': 'law_data_3k.csv', 'row': 605}, page_content='data: 弟弟死亡哥哥可以继承他的房屋吗？如果政府要拆除有没有补偿\n由其第一顺位继承人继承的，如果没有第一顺位继承人的话你才可以继承，第一顺位继承人是指父母子女配偶'),
 Document(id='818344a6-c5ec-42d3-8c45-884abf8695fc', metadata={'source': 'law_data_3k.csv', 'row': 240}, page_content='d

In [ ]:
retriever.invoke("谁可以申请撤销监护人的监护资格？")

[Document(id='f610ec67-0275-4956-8974-0627d97cc62d', metadata={'source': 'law_data_3k.csv', 'row': 501}, page_content='data: 民法典-总则：第三十六条 监护人有下列情形之一的，人民法院根据有关个人或者组织的申请，撤销其监护人资格，安排必要的临时监护措施，并按照最有利于被监护人的原则依法指定监护人：,（一）实施严重损害被监护人身心健康的行为；,（二）怠于履行监护职责，或者无法履行监护职责且拒绝将监护职责部分或者全部委托给他人，导致被监护人处于危困状态；,（三）实施严重侵害被监护人合法权益的其他行为。,本条规定的有关个人、组织包括：其他依法具有监护资格的人，居民委员会、村民委员会、学校、医疗机构、妇女联合会、残疾人联合会、未成年人保护组织、依法设立的老年人组织、民政部门等。,前款规定的个人和民政部门以外的组织未及时向人民法院申请撤销监护人资格的，民政部门应当向人民法院申请。'),
 Document(id='1d28237f-ea4a-4f14-87c5-c66a01e32ceb', metadata={'source': 'law_data_3k.csv', 'row': 287}, page_content='data: 公安部-机动车登记规定2021-12-17：第八十六条 机动车所有人可以委托代理人代理申请各项机动车登记和业务，但共同所有人变更、申请补领机动车登记证书、机动车灭失注销的除外；对机动车所有人因死亡、出境、重病、伤残或者不可抗力等原因不能到场的，可以凭相关证明委托代理人代理申请，或者由继承人申请。,代理人申请机动车登记和业务时，应当提交代理人的身份证明和机动车所有人的委托书。'),
 Document(id='8b8f2d2d-0041-447a-9c4c-b5e2f86e7e94', metadata={'source': 'law_data_3k.csv', 'row': 378}, page_content='data: 市场监督管理总局-互联网药品信息服务管理办法2004-07-08：第二十六条 省、自治区、直辖市食品药品监督管理部门违法对互联网药品信息服务申请作出审核批准的，原发证机关应当撤销原批准的《互联网药品信息服

In [ ]:
retriever.invoke("你现在是一个精通中国法律的法官， 请对以下案件做出分析： 经审理查明： 被告人 xxx 于 2017 年 12 月， 多次在本市 xxx 盗窃财物。 具体事实如下： （一） 2017 年 12 月 9 日 15 时许， 被告人 xxx 在 xxx 店内， 盗窃白色毛衣一件（价值人民币 259 元） 。 现赃物已起获并发还。 （二） 2017 年 12 月 9 日 16 时许， 被告人 xx 在本市 xxx 店内， 盗窃米白色大衣一件（价值人民币 1199 元）。现赃物已起获并发还。 （三） 2017 年 12月 11 日 19 时许， 被告人 xxx 在本市 xxx 内， 盗窃耳机、 手套、 化妆镜等商品共八件（共计价值人民币 357.3 元） 。 现赃物已起获并发还。（四）2017 年 12 月 11 日 20 时许， 被告人 xx 在本市 xxxx 内， 盗窃橙汁、 牛肉干等商品共四件（共计价值人民币 58.39 元） 。 现赃物已起获并发还。2017 年 12 月 11 日， 被告人 xx 被公安机关抓获， 其到案后如实供述了上述犯罪事实。 经鉴定， 被告人 xxx 被诊断为精神分裂症， 限制刑事责任能力， 有受审能力。")

[Document(id='a0476a9f-156a-4df3-bd37-60993d1e29be', metadata={'source': 'law_data_3k.csv', 'row': 1276}, page_content='data: 入室盗窃后敲诈勒索1####元会判刑多久\n表述有问题，不能准确判断，如果只成立盗窃一罪，那么：（一）个人盗窃公私财物“数额较大”，以一千至三千元为起点。处三年以下有期徒刑、拘役或者管制，并处或者单处罚金[3]（二）个人盗窃公私财物“数额巨大”，以三万元至十万元为起点。处三年以上十年以下有期徒刑，并处罚金（三）个人盗窃公私财物“数额特别巨大”，以三十万元至五十万元为起点。处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。各省、自治区、直辖市*级人*法*、人*检**，可以根据该地区经济发展状况，并考虑社会治安状况，参照上列数额，确定该地区执行的数额标准，并报最*人*法*、最*人*检**备案。具体要看你所在地区。'),
 Document(id='b63e6cbd-5c76-4fce-abbe-c88eadfbdce3', metadata={'source': 'law_data_3k.csv', 'row': 1241}, page_content='data: 盗窃青律师可以辩护吗\n尊敬的审判员：我叫###，是本案被告人###的辩护人。辩护人接受指派后依法多次会见了被告人###，详细地查阅了本案全部卷宗材料，对案情有了比较全面、客观的了解。辩护人结合今天的庭审情况，根据质证证据证明的事实与相关法律规定发表如下辩护意见，供法庭审理时参考：一、关于本案的事实，根据我国现行法律关于盗窃罪的规定，以及本案相关证据来看，被告人###已经涉嫌构成盗窃犯罪。同时，鉴于被告人已对所犯罪行供认不讳。因此，辩护人对公诉机关指控###犯盗窃罪无异议。二、辩护人认为本案被告人###具有如下法定和酌定的从轻或者减轻的情节，请合议庭在量刑时予以充分考虑。首先，被告人具有法定的从轻或减轻情节，应减轻处罚。在本案中，被告人XX具有自首情节，且已被公诉机关确认。按照我国刑法第六十七条第一款的规定：对于自首的犯罪嫌疑人，可以从轻或者减轻处罚。辩护人认为：对本案被告人应减轻处罚。其次，被告人具有酌定的从轻或减轻处罚情节。主要有如下：1、被告人具有悔

In [ ]:
retriever.invoke("你现在是一个精通中国法律的法官， 请对以下案件做出分析：2012年5月1日，原告xxx在被告xxxx购买“玉兔牌”香肠15包，其中价值558.6元的14包香肠已过保质期。xxx到收银台结账后，即径直到服务台索赔，后因协商未果诉至法院，要求xxxx店支付14包香肠售价十倍的赔偿金5586元。")

[Document(id='f29f0dba-29fb-4210-a4c6-a9c815f834d2', metadata={'source': 'law_data_3k.csv', 'row': 74}, page_content='data: 民法商法-消费者权益保护法2013-10-25：第五十五条 经营者提供商品或者服务有欺诈行为的，应当按照消费者的要求增加赔偿其受到的损失，增加赔偿的金额为消费者购买商品的价款或者接受服务的费用的三倍；增加赔偿的金额不足五百元的，为五百元。法律另有规定的，依照其规定。,经营者明知商品或者服务存在缺陷，仍然向消费者提供，造成消费者或者其他受害人死亡或者健康严重损害的，受害人有权要求经营者依照本法第四十九条、第五十一条等法律规定赔偿损失，并有权要求所受损失二倍以下的惩罚性赔偿。'),
 Document(id='3bc48820-2b11-4f34-b7d3-45647ead7c2c', metadata={'source': 'law_data_3k.csv', 'row': 1279}, page_content='data: 盗窃罪金额达1####请律师打官司要多少钱\n刑事案件请一个律师大概需要多少钱?具体要看律师事务所的收费，如：(一)刑事案件收费按照各办案阶段分别计件确定收费标准。1、侦查阶段，每件收费2###—1####元。2、审查起诉阶段，每件收费2###—1####元。3、一审阶段，每件收费4###—3####元。4、上述收费标准下浮不限。(二)二审、死刑复核、再审、申诉案件以及刑事自诉案件按照一审阶段的收费标准收取律师服务费。(三)一个律师事务所代理一个案件的多个阶段，自第二阶段起酌减收费。(四)被害人提起刑事附带民事诉讼案件的，按照民事诉讼案件收费标准收取律师服务费。(五)犯罪嫌疑人、被告人同时涉及几个罪名或者数起犯罪事实的，可按照所涉罪名或犯罪事实分别计件收取。'),
 Document(id='cf19b73f-391d-436c-ba41-8e27e2e82354', metadata={'source': 'law_data_3k.csv', 'row': 790}, page_content='data: 我在玖富超能分期了一个手机原价2900多，还17期总共3993，现在微信不能还了，我不还会怎

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise
Context:  {context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("借款人去世， 继承人是否应履行偿还义务？")

'继承人应当履行借款人的偿还债务义务。根据民法典-继承编，如果遗嘱继承或者遗赠附有义务，继承人或者受遗赠人应当履行这些义务。在被保险人去世后，保险金作为被保险人的遗产，保险人有义务按照继承法规定给付保险金。'

In [ ]:
rag_chain.invoke("如何通过法律手段应对民间借贷纠纷？")

'根据最高人民法院相关规定，民间借贷属于民事行为，受到民法和合同法的约束和保护。如果借贷双方约定的利率未超过年利率24%，出借人有权请求借款人按照约定的利率支付利息。超过年利率36%部分的利息则应当被认定无效。'

In [ ]:
rag_chain.invoke("没有赡养老人就无法继承财产吗？")

'没有，根据相关法律规定，子女有赡养父母的义务，但不履行赡养义务不会影响对遗产的继承权。赡养义务和继承权是两个独立的法律概念。'

In [ ]:
rag_chain.invoke("谁可以申请撤销监护人的监护资格？")

'其他依法具有监护资格的人，居民委员会、村民委员会、学校、医疗机构、妇女联合会、残疾人联合会、未成年人保护组织、依法设立的老年人组织、民政部门等可以申请撤销监护人的监护资格。'

In [ ]:
rag_chain.invoke("你现在是一个精通中国法律的法官，请对以下案件做出分析：经审理查明：被告人xxx于2017年12月，多次在本市xxx盗窃财物。具体事实如下：（一）2017年12月9日15时许，被告人xxx在xxx店内，盗窃白色毛衣一件（价值人民币259元）。现赃物已起获并发还。（二）2017年12月9日16时许，被告人xx在本市xxx店内，盗窃米白色大衣一件（价值人民币1199元）。现赃物已起获并发还。（三）2017年12月11日19时许，被告人xxx在本市xxx内，盗窃耳机、手套、化妆镜等商品共八件（共计价值人民币357.3元）。现赃物已起获并发还。（四）2017年12月11日20时许，被告人xx在本市xxxx内，盗窃橙汁、牛肉干等商品共四件（共计价值人民币58.39元）。现赃物已起获并发还。2017年12月11日，被告人xx被公安机关抓获，其到案后如实供述了上述犯罪事实。经鉴定，被告人xxx被诊断为精神分裂症，限制刑事责任能力，有受审能力。")

'根据上述法律规定，被告人xxx的行为构成多次盗窃罪。根据具体案情和被告人的精神分裂症诊断，可能会考虑其限制刑事责任能力。具体判决结果需根据法庭审理的情况综合考量。'

In [ ]:
rag_chain.invoke("你现在是一个精通中国法律的法官， 请对以下案件做出分析：2012年5月1日，原告xxx在被告xxxx购买“玉兔牌”香肠15包，其中价值558.6元的14包香肠已过保质期。xxx到收银台结账后，即径直到服务台索赔，后因协商未果诉至法院，要求xxxx店支付14包香肠售价十倍的赔偿金5586元。")

'根据《消费者权益保护法》第五十五条的规定，经营者提供过期商品属于欺诈行为，应按三倍商品价款赔偿。在此案例中，原告购买的商品14包已过保质期，属于欺诈行为，因此被告店铺需承担法律责任。根据法律规定，原告有权要求赔偿商品售价为三倍，即1675.2元，而非所要求的5586元。'

In [ ]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
import re
from typing import Annotated, Iterator, Literal, TypedDict

from langchain import hub

# llm
# from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

# tool,
# https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/
# TAVILY_API_KEY
from langchain_community.tools.tavily_search import TavilySearchResults

# rag
from langchain_community.document_loaders import web_base
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_core.retrievers import BaseRetriever

# messages & prompts
from langchain_core.messages import BaseMessage, AIMessage, convert_to_messages
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_text_splitters import RecursiveCharacterTextSplitter

# langgraph
from langgraph.graph import END, StateGraph, add_messages

from langgraph.checkpoint.memory import MemorySaver

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
os.environ["TAVILY_API_KEY"] = "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"

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
tavily_search_tool = TavilySearchResults(max_results=3)

In [ ]:
class GraphState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    question: str
    documents: list[Document]
    candidate_answer: str
    retries: int
    web_fallback: bool


class GraphConfig(TypedDict):
    max_retries: int

In [ ]:
MAX_RETRIES = 3
VERBOSE = True

In [ ]:
def document_search(state: GraphState):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    if VERBOSE:
        print("---RETRIEVE---")

    question = convert_to_messages(state["messages"])[-1].content

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question, "web_fallback": True}

In [ ]:
RAG_PROMPT: ChatPromptTemplate = hub.pull("rlm/rag-prompt")
RAG_PROMPT

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
def generate(state: GraphState):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    if VERBOSE:
        print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    retries = state["retries"] if state.get("retries") is not None else -1

    # lcel
    rag_chain = RAG_PROMPT | llm | StrOutputParser()
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"retries": retries + 1, "candidate_answer": generation}

In [ ]:
QUERY_REWRITER_SYSTEM = (
"""
You are a question re-writer that converts an input question to a better version that is optimized for vectorstore retrieval.
Look at the input and try to reason about the underlying semantic intent / meaning.
"""
)

QUERY_REWRITER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", QUERY_REWRITER_SYSTEM),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

In [ ]:
def transform_query(state: GraphState):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """
    if VERBOSE:
        print("---TRANSFORM QUERY---")

    question = state["question"]

    # Re-write question
    query_rewriter = QUERY_REWRITER_PROMPT | llm | StrOutputParser()
    better_question = query_rewriter.invoke({"question": question})
    return {"question": better_question}

In [ ]:
def web_search(state: GraphState):
    if VERBOSE:
        print("---RUNNING WEB SEARCH---")

    question = state["question"]
    documents = state["documents"]
    print("---RUNNING TRAVILY SEARCH---")
    search_results = tavily_search_tool.invoke(question)
    print(search_results)
    search_content = "\n".join([d["content"] for d in search_results])
    documents.append(Document(page_content=search_content, metadata={"source": "websearch"}))
    return {"documents": documents, "web_fallback": False}

In [ ]:
def finalize_response(state: GraphState):
    if VERBOSE:
        print("---FINALIZING THE RESPONSE---")

    return {"messages": [AIMessage(content=state["candidate_answer"])]}

In [ ]:
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


HALLUCINATION_GRADER_SYSTEM = (
"""
You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts.
Give a binary score 'yes' or 'no', where 'yes' means that the answer is grounded in / supported by the set of facts.

IF the generation includes code examples, make sure those examples are FULLY present in the set of facts, otherwise always return score 'no'.
"""
)

HALLUCINATION_GRADER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", HALLUCINATION_GRADER_SYSTEM),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)


class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


ANSWER_GRADER_SYSTEM = (
"""
You are a grader assessing whether an answer addresses / resolves a question.
Give a binary score 'yes' or 'no', where 'yes' means that the answer resolves the question.
"""
)

ANSWER_GRADER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", ANSWER_GRADER_SYSTEM),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

In [ ]:
def grade_generation_v_documents_and_question(state: GraphState, config) -> Literal["generate", "transform_query", "web_search", "finalize_response"]:
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """
    question = state["question"]
    documents = state["documents"]
    generation = state["candidate_answer"]
    web_fallback = state["web_fallback"]
    retries = state["retries"] if state.get("retries") is not None else -1
    max_retries = config.get("configurable", {}).get("max_retries", MAX_RETRIES)

    # this means we've already gone through web fallback and can return to the user
    if not web_fallback:
        return "finalize_response"

    if VERBOSE:
        print("---CHECK HALLUCINATIONS---")

    # llm lcel chain
    hallucination_grader = HALLUCINATION_GRADER_PROMPT | llm.with_structured_output(GradeHallucinations)
    hallucination_grade: GradeHallucinations = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )

    # Check hallucination
    if hallucination_grade.binary_score == "no":
        if VERBOSE: print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "generate" if retries < max_retries else "web_search"

    if VERBOSE:
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        print("---GRADE GENERATION vs QUESTION---")

    # Check question-answering
    answer_grader = ANSWER_GRADER_PROMPT | llm.with_structured_output(GradeAnswer)
    answer_grade: GradeAnswer = answer_grader.invoke({"question": question, "generation": generation})
    if answer_grade.binary_score == "yes":
        if VERBOSE: print("---DECISION: GENERATION ADDRESSES QUESTION---")
        return "finalize_response"
    else:
        if VERBOSE: print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
        return "transform_query" if retries < max_retries else "web_search"

In [ ]:
workflow = StateGraph(GraphState, config_schema=GraphConfig)

# Define the nodes
workflow.add_node("document_search", document_search)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)
workflow.add_node("web_search", web_search)
workflow.add_node("finalize_response", finalize_response)

# Build graph
workflow.set_entry_point("document_search")
workflow.add_edge("document_search", "generate")
workflow.add_edge("transform_query", "document_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("finalize_response", END)

workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question
)

# Compile
graph = workflow.compile()

In [ ]:
VERBOSE = True
inputs = {"messages": [("你现在是一个精通中国法律的法官， 请对以下案件做出分析：2012年5月1日，原告xxx在被告xxxx购买“玉兔牌”香肠15包，其中价值558.6元的14包香肠已过保质期。xxx到收银台结账后，即径直到服务台索赔，后因协商未果诉至法院，要求xxxx店支付14包香肠售价十倍的赔偿金5586元。")]}
for output in graph.stream(inputs):
    print(output)
    print("\n---\n")

---RETRIEVE---
{'document_search': {'documents': [Document(metadata={'row': 74, 'source': 'law_data_3k.csv'}, page_content='data: 民法商法-消费者权益保护法2013-10-25：第五十五条 经营者提供商品或者服务有欺诈行为的，应当按照消费者的要求增加赔偿其受到的损失，增加赔偿的金额为消费者购买商品的价款或者接受服务的费用的三倍；增加赔偿的金额不足五百元的，为五百元。法律另有规定的，依照其规定。,经营者明知商品或者服务存在缺陷，仍然向消费者提供，造成消费者或者其他受害人死亡或者健康严重损害的，受害人有权要求经营者依照本法第四十九条、第五十一条等法律规定赔偿损失，并有权要求所受损失二倍以下的惩罚性赔偿。'), Document(metadata={'row': 77, 'source': 'law_data_3k.csv'}, page_content='data: 民法商法-消费者权益保护法2013-10-25：第五十八条 经营者违反本法规定，应当承担民事赔偿责任和缴纳罚款、罚金，其财产不足以同时支付的，先承担民事赔偿责任。'), Document(metadata={'row': 62, 'source': 'law_data_3k.csv'}, page_content='data: 民法商法-消费者权益保护法2013-10-25：第四十三条 消费者在展销会、租赁柜台购买商品或者接受服务，其合法权益受到损害的，可以向销售者或者服务者要求赔偿。展销会结束或者柜台租赁期满后，也可以向展销会的举办者、柜台的出租者要求赔偿。展销会的举办者、柜台的出租者赔偿后，有权向销售者或者服务者追偿。'), Document(metadata={'row': 1101, 'source': 'law_data_3k.csv'}, page_content='data: 江*赣*章*区饿了么外卖拖欠骑手工资，广大消费者\n拖欠工资，不发工资、克扣工资可以打12###电话投诉，该电话是人力资源和社会保障局的电话。也可以到劳*局****投诉。由于公司未能按时发放劳*报酬，当事人可以申请解除劳*合同和劳*关系，并要求经济补偿。当事人可以向当地劳***行*部*进行举报

/usr/local/lib/python3.10/dist-packages/langchain_openai/chat_models/base.py:1354: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


/usr/local/lib/python3.10/dist-packages/langchain_openai/chat_models/base.py:1354: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


---DECISION: GENERATION ADDRESSES QUESTION---
{'generate': {'retries': 0, 'candidate_answer': '根据《消费者权益保护法》第五十五条，经营者提供过期商品属于欺诈行为，消费者有权要求增加赔偿。增加赔偿的金额为消费者购买商品的价款的三倍，但不足五百元的，为五百元。因此，原告可以要求三倍赔偿，即558.6元的三倍为1675.8元，而不是十倍赔偿。'}}

---

---FINALIZING THE RESPONSE---
{'finalize_response': {'messages': [AIMessage(content='根据《消费者权益保护法》第五十五条，经营者提供过期商品属于欺诈行为，消费者有权要求增加赔偿。增加赔偿的金额为消费者购买商品的价款的三倍，但不足五百元的，为五百元。因此，原告可以要求三倍赔偿，即558.6元的三倍为1675.8元，而不是十倍赔偿。', additional_kwargs={}, response_metadata={}, id='f2193137-30ba-4bbc-b2d6-b28808103543')]}}

---

